In [1]:
library(ggplot2)
library(dplyr)
library(readxl)
oldw <- getOption("warn")
options(warn = -1)
#options(warn = oldw)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'readxl' was built under R version 3.6.3"

In [2]:
# FOR POPULATION DATA

population<- read.csv("ARMMPopulation2010-2015.csv")

#Get the sum for each age distribution
sumpoplist<-NULL
population[population==NA]<-0

#get for ages 0to 60
for (j in 0:5){
    sumpoplist2<-NULL
    for (i in 1:nrow(population)){
        poplist<-NULL
        for (l in 1:10){
            n<- j*10+l
            if (n==1){
                poplist<-c(poplist,as.numeric(paste(population[i,"sin_age_bot_und_001"]))) #to include less than 1 year old
            }
            if (n<10){
                age <- paste0("sin_age_bot_00",n) #for formatting of those lessthan 10 years old
            }
            if (n>=10){
                age <- paste0("sin_age_bot_0",n) #for greater than 10 years old
            }
            poplist<- c(poplist,as.numeric(paste(population[i,grep(age,colnames(population))]))) 
        }
    sumpoplist2<- c(sumpoplist2,sum(poplist))
    }
    denote<- paste0("age_cut",j)
    population[denote]<- sumpoplist2 #will have for each 10 years a secific column, ex:age_cut0 means 0-10 years old
}

# for 60 and above
for (i in 1:nrow(population)){
    poplist<-NULL
    for (k in grep("sin_age_bot_060",colnames(population)):grep("sin_age_bot_080_and_ove",colnames(population))){
        poplist<-c(poplist,as.numeric(paste(population[i,k])))
    }
    sumpoplist<-c(sumpoplist,sum(poplist)) #put na.rm=TRUE if you want value to be 0 for NA
}
population['vulpop']<-sumpoplist

         
#get the arranged population
arrangedpopulation<- population %>% 
         group_by(PSGC_CITY.MUNI) %>% 
         summarise(ProviCode=first(PSGC_PROV),Region=first(REGI),Municipality=first(MuniCities),Pop2015=sum(Pop2015),HHnum=sum(as.numeric(paste(num_hh0))),Malepop=sum(as.numeric(paste(tot_sin_age_mal))),Femalepop=sum(as.numeric(paste(tot_sin_age_fem))),Age_0_10=sum(age_cut0),Age_11_20=sum(age_cut1),Age_21_30=sum(age_cut2),Age_31_40=sum(age_cut3),Age_41_50=sum(age_cut4),Age_51_60=sum(age_cut5),vulnerable=sum(vulpop)) %>% distinct()

colnames(arrangedpopulation)[1]<- "MuniCode"

#adding density into the dataset
municipalityAreas<- read_excel("mun_areas.xlsx",sheet=1)
colnames(municipalityAreas)[1]<-"MuniCode"
arrangedpopulation<-Reduce(function(x, y) merge(x, y,by="MuniCode",all=TRUE), list(arrangedpopulation, municipalityAreas))
arrangedpopulation$populationDensity<- arrangedpopulation$Pop2015/arrangedpopulation$Area
head(arrangedpopulation)

write.csv(arrangedpopulation,'ProcessedPopulation2015.csv')

MuniCode,ProviCode,Region,Municipality,Pop2015,HHnum,Malepop,Femalepop,Age_0_10,Age_11_20,Age_21_30,Age_31_40,Age_41_50,Age_51_60,vulnerable,Area,populationDensity
PH099701000,PH099700000,Zamboanga Peninsula,City Of Isabela (Not A Province),112788,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
PH103502000,PH103500000,Northern Mindanao,Baloi,58383,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
PH103514000,PH103500000,Northern Mindanao,Munai,32973,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
PH103515000,PH103500000,Northern Mindanao,Nunungan,18367,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
PH103520000,PH103500000,Northern Mindanao,Tagoloan,13253,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
PH103521000,PH103500000,Northern Mindanao,Tangcal,15181,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [3]:
#Adding schools to the datalist
schools<- read.csv("SchoolsperMuni.csv")
schools<- schools %>% select(MuniCode=PSGC_CITY.MUNI,NumSchools=Number.of.School)
arrangedDataset<- merge(arrangedpopulation,schools,by="MuniCode",all="TRUE")
infrastractureDataset<-arrangedDataset

#to get a list of all the municipality codes for the future
MuniCodes<- NULL
for (i in 1:nrow(arrangedDataset)){
    MuniCodes<- c(MuniCodes,as.character(arrangedDataset[i,1]))
}
MuniData<- data.frame(MuniCodes=MuniCodes,stringsAsFactors=FALSE)
write.csv(MuniData,"MunicipalityCodes.csv")

#to get a list of municipality names in the future
Municipalities<- NULL
for (i in 1:nrow(arrangedDataset)){
    Municipalities<- c(Municipalities,as.character(arrangedDataset[i,4]))
}


write.csv(arrangedDataset,"processedData.csv")

In [4]:
#Filter data based on BARMM only
covidData<- read.csv('Coviddata.csv')
covidData<- filter(covidData, CityMuniPSGC %in% MuniCodes)
write.csv(covidData,"Coviddata.csv")

#save data for future purposes
newCovidData<-covidData[as.character(covidData$RemovalType)=='',]
write.csv(covidData,"newCoviddata.csv")

In [5]:
# FOR COVID DATASET

#get covid 19 related features (see CODEBOOK for details) for each municipality in BARMM
arrangedCovidData<-covidData %>% mutate(current=ifelse(as.character(covidData$RemovalType)=='',1,0),recovered=ifelse(as.character(covidData$RemovalType)=='RECOVERED',1,0),died=ifelse(as.character(covidData$RemovalType)=='DIED',1,0),totalcase=1)
arrangedCovidData<- arrangedCovidData %>% group_by(CityMuniPSGC) %>% summarise(currentCovid=sum(current),recovered=sum(recovered),died=sum(died),totalcase=sum(totalcase)) %>% distinct() %>% mutate(CaseFatalRate=died/totalcase,RecoverRate=recovered/totalcase)
colnames(arrangedCovidData)[1]<- "MuniCode"
write.csv(arrangedCovidData,"pathologicalData.csv")

arrangedDataset1<-Reduce(function(x, y) merge(x, y,by="MuniCode",all=TRUE), list(arrangedDataset, arrangedCovidData))

In [6]:
# FOR ECONOMIC DATA

# get economic data for each municipality
economicData<- read_excel('Economic-Factors-Complete.xlsx')
colnames(economicData)[4]<-"MuniCode"
economicData<- economicData %>% select(-c(colnames(economicData)[1:3]))

arrangedDataset2<-Reduce(function(x, y) merge(x, y,by="MuniCode",all=TRUE), list(arrangedDataset1, economicData))

In [7]:
# For Health Infrastracture
#health workers data
healthProfData<- read.csv("healthcareProfessionals.csv")
colnames(healthProfData)[6]<-"MuniCode"
healthProfData$totalHealthProf <- rowSums(healthProfData[,8:10])
healthProfData<- healthProfData %>% group_by(MuniCode) %>% summarise(healthProfessionals=sum(totalHealthProf)) %>% distinct()
write.csv(healthProfData,"healthWorkers.csv")

# bed Data
bedData<- read.csv("hospitalBeds.csv")
colnames(bedData)[12]<- "ProviCode"
bedData$totalBeds<- rowSums(bedData[,5:8])
bedData$occupiedBeds<- rowSums(bedData[,c(6,8)])
bedData<- bedData %>% group_by(ProviCode) %>% summarise(numberOfBeds=sum(totalBeds,na.rm=TRUE),numberOfOccupiedBeds=sum(occupiedBeds,na.rm=TRUE),occupancyRate=sum(occupiedBeds,na.rm=TRUE)/sum(totalBeds,na.rm=TRUE))
write.csv(bedData,"provincialBeds.csv")

# RHU and BHS data
rhubhsData<- read_excel("RHUBHS.xlsx")
colnames(rhubhsData)[1]<- "MuniCode"
rhubhsData[,5]<- as.numeric(unlist(rhubhsData[,5]))
rhubhsData[,6]<- as.numeric(unlist(rhubhsData[,6]))
rhubhsData$totalUnits<- rowSums(rhubhsData[,5:6])
rhubhsData<- rhubhsData %>% select(MuniCode,totalUnits)
write.csv(rhubhsData,"RHUandBHSData.csv")

#merging the data
arrangedDataset2<-Reduce(function(x, y) merge(x, y,by="MuniCode",all=TRUE), list(arrangedDataset2, rhubhsData))
arrangedDataset2<-Reduce(function(x, y) merge(x, y,by="MuniCode",all=TRUE), list(arrangedDataset2, healthProfData))
arrangedDataset2<-Reduce(function(x, y) merge(x, y,by="ProviCode",all=TRUE), list(arrangedDataset2, bedData))

In [8]:
# FOR HOUSEHOLD DATA

#household data to merge with population and aranngedDataset
householdData<- data.frame(read_excel("Table-2_8.xlsx")[,-2])
householdData<- householdData[complete.cases(householdData),]
colnames(householdData)<- c('Municipality',householdData[1,-1])
for (i in 1:nrow(householdData)){
    householdData[i,1]<- stringr::str_to_title(householdData[i,1])
    }

#to get only the BARMM data for Household Data + preprocessing
householdData<- filter(householdData,Municipality %in% Municipalities)
householdData[householdData$Municipality=="Carmen",]<- householdData[householdData$Municipality=="Carmen" & householdData["Household Population"]=="95865",]
householdData[householdData$Municipality=="Tagoloan",]<- householdData[householdData$Municipality=="Tagoloan" & householdData["Household Population"]=="13229",]
householdData[householdData$Municipality=="Tuburan",]<- householdData[householdData$Municipality=="Tuburan" & householdData["Household Population"]=="20207",]
householdData[householdData$Municipality=="Kapatagan",]<- householdData[householdData$Municipality=="Kapatagan" & householdData["Household Population"]=="15487",]
householdData<- householdData %>% distinct()
householdData[,"Average Household Size"]<- as.numeric(unlist(householdData[,"Average Household Size"]))
write.csv(householdData,"householdData.csv")

#to merge household Data to the whole dataframe
arrangedDataset3<-Reduce(function(x, y) merge(x, y,by="Municipality",all=TRUE), list(arrangedDataset2, householdData))
arrangedDataset3<- arrangedDataset3[-c(87,90),] #remove repeating data

New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5


In [9]:
# creating a function for normalization
normalize <-function (x){
    (x-range(x,na.rm=TRUE)[1])/(range(x,na.rm=TRUE)[2]-range(x,na.rm=TRUE)[1])
}

#Normalization but those with higher values have lower ndex
normalize2 <-function (x){
    (range(x,na.rm=TRUE)[2]-x)/(range(x,na.rm=TRUE)[2]-range(x,na.rm=TRUE)[1])
}

#creating a new column for the normalized values of each feature needed
arrangedDataset4<- arrangedDataset3 %>% select("Municipality","MuniCode")

#Demographic Data
arrangedDataset3$vulnerableratio<- arrangedDataset3$vulnerable/arrangedDataset3$Pop2015
arrangedDataset4$vulnerableScore<- normalize(arrangedDataset3$vulnerableratio)
arrangedDataset4$averageHouseScore<- normalize(arrangedDataset3[,"Average Household Size"])
arrangedDataset4$popDensityScore<- normalize(arrangedDataset3$populationDensity)
arrangedDataset4$demographicScore<- rowSums(arrangedDataset4[,3:5],na.rm=TRUE)

#Financial Data
#Merging economic Dataset with ranks (done on another code)
povertyData<- read.csv("povertyrank.csv")
colnames(povertyData)[1]<- "MuniCode"
arrangedDataset4<-Reduce(function(x, y) merge(x, y,by="MuniCode",all=TRUE), list(arrangedDataset4, povertyData))
arrangedDataset4$Poverty.Incidence_2015<- NULL
arrangedDataset4$Rank<- NULL
colnames(arrangedDataset4)[7]<- "povertyScore"
arrangedDataset4$finConScore<- normalize2(arrangedDataset3$EQUITY)
arrangedDataset4$cashPosScore<- normalize2(arrangedDataset3$SURPLUS)
arrangedDataset4$availFundScore<- normalize2(arrangedDataset3$SURPLUS/arrangedDataset3$Pop2015)
arrangedDataset4$financialScore<- rowSums(arrangedDataset4[,7:10],na.rm=TRUE)

# Health Infrastracture Factors
arrangedDataset3$rhubhsratio<- arrangedDataset3$totalUnits/arrangedDataset3$Pop2015
arrangedDataset3$healthworkersratio<- arrangedDataset3$healthProfessionals/arrangedDataset3$Pop2015
arrangedDataset3<- arrangedDataset3 %>% group_by(ProviCode) %>% mutate(bedratio=first(arrangedDataset3$numberOfBeds)/sum(arrangedDataset3$Pop2015,na.rm=TRUE))
arrangedDataset4$rhubhsScore<- normalize2(arrangedDataset3$rhubhsratio)
arrangedDataset4$healthWorkerScore<- normalize2(arrangedDataset3$healthworkersratio)
arrangedDataset4$numberOfBedScore<- normalize2(arrangedDataset3$bedratio)
arrangedDataset4$healthInfrastractureScore<- rowSums(arrangedDataset4[,12:14],na.rm=TRUE)

#Epidemiological Factors
arrangedDataset4$totalCaseScore<- normalize(arrangedDataset3$totalcase/arrangedDataset3$Pop2015)
arrangedDataset4$activeCaseScore<- normalize(arrangedDataset3$currentCovid/arrangedDataset3$Pop2015)
arrangedDataset4$caseFatalScore<- arrangedDataset3$CaseFatalRate
arrangedDataset4$epidomiologicalScore<- rowSums(arrangedDataset4[,16:18],na.rm=TRUE)

#getting total index score
arrangedDataset4$totalScore<- rowSums(arrangedDataset4[,c(6,11,15,19)],na.rm=TRUE)

#Ranking the data
#Since we normalized the values, the range will only go from 0 to 1. We convert all NA to -1 for the sake of ranking issues
#since we wanted to make all NA's have the same ranking. After ranking, we convert all -1 back to NA.
arrangedDataset4[arrangedDataset4==NA]<--1
#arrangedDataset4$vulnerableRank<- rank(-arrangedDataset4$vulnerableScore,na.last=TRUE,ties.method="min")
#arrangedDataset4$averageHouseRank<- rank(-arrangedDataset4$averageHouseScore,na.last=TRUE,ties.method="min")
#arrangedDataset4$finConRank<- rank(-arrangedDataset4$finConScore,na.last=TRUE,ties.method="min")
#arrangedDataset4$cashPosRank<- rank(-arrangedDataset4$cashPosScore,na.last=TRUE,ties.method="min")
#arrangedDataset4$availFundRank<- rank(-arrangedDataset4$availFundScore,na.last=TRUE,ties.method="min")
#arrangedDataset4$totalCaseRank<- rank(-arrangedDataset4$totalCaseScore,na.last=TRUE,ties.method="min")
#arrangedDataset4$caseFatalRank<- rank(-arrangedDataset4$caseFatalScore,na.last=TRUE,ties.method="min")
#arrangedDataset4$totalScoreRank<- rank(-arrangedDataset4$totalScore,na.last=TRUE,ties.method="min")

arrangedDataset4$demographicRank<- rank(-arrangedDataset4$demographicScore,na.last=TRUE,ties.method="min")
arrangedDataset4$financialRank<- rank(-arrangedDataset4$financialScore,na.last=TRUE,ties.method="min")
arrangedDataset4$healthInfrastractureRank<- rank(-arrangedDataset4$healthInfrastractureScore,na.last=TRUE,ties.method="min")
arrangedDataset4$epidomiologicalRank<- rank(-arrangedDataset4$epidomiologicalScore,na.last=TRUE,ties.method="min")
arrangedDataset4$totalScoreRank<- rank(-arrangedDataset4$totalScore,na.last=TRUE,ties.method="min")
arrangedDataset4[arrangedDataset4==-1]<-NA

#Saving the data
write.csv(arrangedDataset4,"rankingData.csv")

In [10]:
head(arrangedDataset4,20)

MuniCode,Municipality,vulnerableScore,averageHouseScore,popDensityScore,demographicScore,povertyScore,finConScore,cashPosScore,availFundScore,...,totalCaseScore,activeCaseScore,caseFatalScore,epidomiologicalScore,totalScore,demographicRank,financialRank,healthInfrastractureRank,epidomiologicalRank,totalScoreRank
PH099701000,City Of Isabela (Not A Province),NA,NA,NA,0.000000000,0.000,0.9639628,0.9062376,0.9141016,...,0.016382159,0.000000000,0.000000000,0.016382159,4.3430323,132,97,48,84,106
PH103502000,Baloi,NA,0.46748350,NA,0.467483500,0.325,0.9975619,0.9084763,0.9177147,...,0.017333417,0.000000000,0.000000000,0.017333417,5.3373380,117,76,29,83,58
PH103514000,Munai,NA,0.37301064,NA,0.373010636,0.611,NA,NA,NA,...,0.144406069,0.043200543,0.090909091,0.278515702,1.2625263,119,120,118,10,130
PH103515000,Nunungan,NA,0.30678982,NA,0.306789818,0.838,0.9562227,0.8328033,0.8250672,...,0.038473160,0.022411120,0.000000000,0.060884280,5.2266112,121,24,56,49,62
PH103520000,Tagoloan,NA,0.17045744,NA,0.170457444,0.677,0.9036785,0.9041743,0.9114651,...,0.029833835,0.013334729,0.125000000,0.168168564,4.2528448,125,37,116,23,108
PH103521000,Tangcal,NA,0.62792793,NA,0.627927927,0.782,0.9630411,0.8365043,0.8361194,...,0.044992310,0.020724791,0.066666667,0.132383768,4.8941949,111,33,92,33,83
PH103523000,Pantar,NA,0.64057655,NA,0.640576550,0.557,0.9443507,0.9152673,0.9243887,...,0.126948949,0.009523958,0.045454545,0.181927452,4.7559577,110,46,107,20,88
PH124702000,Carmen,NA,0.05013877,0.0010326780,0.051171451,0.541,NA,NA,NA,...,0.159454878,0.000000000,0.009090909,0.168545787,0.7607172,129,128,118,22,132
PH124703000,Kabacan,NA,0.05202128,0.0006275864,0.052648865,0.267,0.6306199,0.0000000,0.0000000,...,0.022585022,0.000000000,0.000000000,0.022585022,1.5491792,128,112,110,77,126
PH124709000,Midsayap,NA,0.06032987,0.0085469808,0.068876848,0.309,0.9758484,0.9088168,0.9179756,...,0.032721001,0.000000000,0.000000000,0.032721001,4.9672377,127,81,20,68,78
